## Imports of libraries and additional datasets

In [1]:
# Import epfl google drive!
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
# Change to shared project drive
%cd /content/drive/Shareddrives/ADA-project

/content/drive/Shareddrives/ADA-project


In [37]:
import pandas as pd
import numpy as np
import bz2
import json

In [4]:
# Speaker attributes in df
df = pd.read_parquet('Project datasets/speaker_attributes.parquet')
df.head()

,aliases,date_of_birth,nationality,gender,lastrevid,ethnic_group,US_congress_bio_ID,occupation,party,academic_degree,id,label,candidacy,type,religion
0,"[Washington, President Washington, G. Washingt...",[+1732-02-22T00:00:00Z],"[Q161885, Q30]",[Q6581097],1395141751,None,W000178,"[Q82955, Q189290, Q131512, Q1734662, Q294126, ...",[Q327591],None,Q23,George Washington,"[Q698073, Q697949]",item,[Q682443]
1,"[Douglas Noel Adams, Douglas Noël Adams, Dougl...",[+1952-03-11T00:00:00Z],[Q145],[Q6581097],1395737157,[Q7994501],None,"[Q214917, Q28389, Q6625963, Q4853732, Q1884422...",None,None,Q42,Douglas Adams,None,item,None
2,"[Paul Marie Ghislain Otlet, Paul Marie Otlet]",[+1868-08-23T00:00:00Z],[Q31],[Q6581097],1380367296,None,None,"[Q36180, Q40348, Q182436, Q1265807, Q205375, Q...",None,None,Q1868,Paul Otlet,None,item,None
3,"[George Walker Bush, Bush Jr., Dubya, GWB, Bus...",[+1946-07-06T00:00:00Z],[Q30],[Q6581097],1395142029,None,None,"[Q82955, Q15982858, Q18814623, Q1028181, Q1408...",[Q29468],None,Q207,George W. Bush,"[Q327959, Q464075, Q3586276, Q4450587]",item,"[Q329646, Q682443, Q33203]"
4,"[Velázquez, Diego Rodríguez de Silva y Velázqu...",[+1599-06-06T00:00:00Z],[Q29],[Q6581097],1391704596,None,None,[Q1028181],None,None,Q297,Diego Velázquez,None,item,None


In [ ]:
!python Project\ datasets/query_wikidata_dump.py --input /content/drive/MyDrive/wikidata-20211004-all.json.gz  --output speakers.json

Traceback (most recent call last):
  File "Project datasets/query_wikidata_dump.py", line 190, in <module>
    main()
  File "Project datasets/query_wikidata_dump.py", line 22, in main
    with gzip.open(args.input, 'rb') as s_file:
  File "/usr/lib/python3.7/gzip.py", line 58, in open
    binary_file = GzipFile(filename, gz_mode, compresslevel)
  File "/usr/lib/python3.7/gzip.py", line 168, in __init__
    fileobj = self.myfileobj = builtins.open(filename, mode or 'rb')
FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/wikidata-20211004-all.json.gz'


##Cleaning 

### Plan
*   Descriptive statistics
*   types
*   Missing data
*   distribution (histograms)
*   outliers
*   duplicates
*  inconsistent data (wrong format etc)
*   correlation between parameters
*  filter columns that we don't need
*  maybe create additional ones
*  standardize ?

In [ ]:
#Percentage of lines with null gender : 
df[df.gender.isnull()].size / df.size

0.21536937853557775

In [5]:
#Remove null gender : 
df_2 = df.drop(df[df.gender.isnull()].index)
df_2.head()

,aliases,date_of_birth,nationality,gender,lastrevid,ethnic_group,US_congress_bio_ID,occupation,party,academic_degree,id,label,candidacy,type,religion
0,"[Washington, President Washington, G. Washingt...",[+1732-02-22T00:00:00Z],"[Q161885, Q30]",[Q6581097],1395141751,None,W000178,"[Q82955, Q189290, Q131512, Q1734662, Q294126, ...",[Q327591],None,Q23,George Washington,"[Q698073, Q697949]",item,[Q682443]
1,"[Douglas Noel Adams, Douglas Noël Adams, Dougl...",[+1952-03-11T00:00:00Z],[Q145],[Q6581097],1395737157,[Q7994501],None,"[Q214917, Q28389, Q6625963, Q4853732, Q1884422...",None,None,Q42,Douglas Adams,None,item,None
2,"[Paul Marie Ghislain Otlet, Paul Marie Otlet]",[+1868-08-23T00:00:00Z],[Q31],[Q6581097],1380367296,None,None,"[Q36180, Q40348, Q182436, Q1265807, Q205375, Q...",None,None,Q1868,Paul Otlet,None,item,None
3,"[George Walker Bush, Bush Jr., Dubya, GWB, Bus...",[+1946-07-06T00:00:00Z],[Q30],[Q6581097],1395142029,None,None,"[Q82955, Q15982858, Q18814623, Q1028181, Q1408...",[Q29468],None,Q207,George W. Bush,"[Q327959, Q464075, Q3586276, Q4450587]",item,"[Q329646, Q682443, Q33203]"
4,"[Velázquez, Diego Rodríguez de Silva y Velázqu...",[+1599-06-06T00:00:00Z],[Q29],[Q6581097],1391704596,None,None,[Q1028181],None,None,Q297,Diego Velázquez,None,item,None


In [ ]:
#Duplicates
duplicates = df_2[df_2.duplicated(subset='id', keep='first')] 
duplicates.head()

,aliases,date_of_birth,nationality,gender,lastrevid,ethnic_group,US_congress_bio_ID,occupation,party,academic_degree,id,label,candidacy,type,religion


In [6]:
#Remove unncessary columns
df_3 = df_2.drop(['US_congress_bio_ID', 'party', 'candidacy'], axis=1) 
# drop also aliases, date_of_birth, ethnic_group, label, religion ??

In [34]:
#Percentage of lines with null nationality : 
df_3[df_3.nationality.isnull()].size / df.size

0.2547805698797292

In [7]:
#Percentage of lines with null date of birth : 
df_3[df_3.date_of_birth.isnull()].size / df.size

0.20053990837657457

In [ ]:
#Percentage of lines with null ethnic_group : 
df_3[df_3.ethnic_group.isnull()].size / df.size

0.6163618055294064

In [8]:
#Remove ethnic group column as it has above 50% of null values : 
df_3 = df_3.drop('ethnic_group', axis=1)

In [ ]:
#Percentage of lines with null label : 
df_3[df_3.label.isnull()].size / df.size

0.03188346648106557

In [ ]:
#Percentage of lines with null religion : 
df_3[df_3.religion.isnull()].size / df.size

0.5596987302277541

In [9]:
#Remove religion column as it has above 50% of null values : 
df_3 = df_3.drop('religion', axis=1)

In [88]:
df_3[df_3.aliases.isnull()].size / df.size

0.443215888667022

## Retrieve quotebank wikidata labels 


In [10]:
labels = pd.read_csv('Project datasets/wikidata_labels_descriptions_quotebank.csv.bz2', compression='bz2', index_col='QID')
labels.head()

,Label,Description
QID,,
Q31,Belgium,country in western Europe
Q45,Portugal,country in southwestern Europe
Q75,Internet,global system of connected computer networks
Q148,People's Republic of China,sovereign state in East Asia
Q155,Brazil,country in South America


In [11]:
labels = labels.drop(labels[labels.Label.isnull()].index)
labels.head()

,Label,Description
QID,,
Q31,Belgium,country in western Europe
Q45,Portugal,country in southwestern Europe
Q75,Internet,global system of connected computer networks
Q148,People's Republic of China,sovereign state in East Asia
Q155,Brazil,country in South America


In [62]:
labels.loc['Q6581097']['Label']

'male'

In [70]:
#Replacing gender QUIDs by value
#for index, row in df_3.iterrows():
#  quid = row.loc['gender'][0]
#  row.loc['gender'] = labels.loc[quid]['Label']
#--> Not useful, replace only the ones needed

###Test with the 2019 New York Times Data

In [72]:
df_quotesNY = pd.read_json('quotes-2019-nytimes.json', lines=True)
df_quotesNY.head()

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
0,2019-04-17-024782,"It is not a low-income immigration,",James Fisher,"[Q16213953, Q20707104, Q43143598, Q58886302, Q...",2019-04-17 13:31:18,1,"[[James Fisher, 0.7475], [None, 0.2525]]",[https://www.nytimes.com/2019/04/17/realestate...,E
1,2019-04-02-001128,a champion figure skater switching to roller s...,John Updike,[Q105756],2019-04-02 14:58:33,2,"[[John Updike, 0.5856], [None, 0.4144]]",[https://www.nytimes.com/2019/04/02/opinion/vl...,E
2,2019-05-09-055187,It makes it much more difficult for him to mak...,None,[],2019-05-09 18:11:29,1,"[[None, 0.6493], [President Bill Clinton, 0.27...",[http://mobile.nytimes.com/2019/05/09/world/as...,E
3,2019-10-31-056366,"It puts me in a predicament,",Xavier Becerra,[Q1855840],2019-10-31 16:45:15,3,"[[Xavier Becerra, 0.9065], [None, 0.0909], [St...",[http://www.nytimes.com/2019/10/31/technology/...,E
4,2019-01-04-001792,A Pile of Leaves.,None,[],2019-01-04 10:00:07,1,"[[None, 0.8737], [Jason Fulford, 0.1263]]",[https://www.nytimes.com/2019/01/04/books/revi...,E


In [80]:
#Filter null speaker 
df_quotesNY = df_quotesNY.drop(df_quotesNY[df_quotesNY.speaker == 'None'].index)
df_quotesNY.head()

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
0,2019-04-17-024782,"It is not a low-income immigration,",James Fisher,"[Q16213953, Q20707104, Q43143598, Q58886302, Q...",2019-04-17 13:31:18,1,"[[James Fisher, 0.7475], [None, 0.2525]]",[https://www.nytimes.com/2019/04/17/realestate...,E
1,2019-04-02-001128,a champion figure skater switching to roller s...,John Updike,[Q105756],2019-04-02 14:58:33,2,"[[John Updike, 0.5856], [None, 0.4144]]",[https://www.nytimes.com/2019/04/02/opinion/vl...,E
3,2019-10-31-056366,"It puts me in a predicament,",Xavier Becerra,[Q1855840],2019-10-31 16:45:15,3,"[[Xavier Becerra, 0.9065], [None, 0.0909], [St...",[http://www.nytimes.com/2019/10/31/technology/...,E
5,2019-08-15-002017,A Senator we can call our own.,Tom Rath,[Q7817334],2019-08-15 22:36:33,1,"[[Tom Rath, 0.7598], [None, 0.1993], [Warren R...",[http://www.nytimes.com/2019/08/15/us/politics...,E
8,2019-07-22-032883,"It's a success, a relief and a technical feat,",Florence Parly,[Q3074013],2019-07-22 02:37:50,21,"[[Florence Parly, 0.9262], [None, 0.0738]]",[http://www.breitbart.com/news/french-submarin...,E


In [103]:
# Retrieve New York times speakers' gender
speakerGender = []
for NY_index, NY_row in df_quotesNY.iterrows():
  q_speaker = NY_row.loc['qids'] 
  #qids are not of the same people for each quotation
  #40% of speakers do not have aliases, how to link both?
  if df_3.id.isin(q_speaker).any(): 
    genders = []
    for q in q_speaker:
      if q in df_3.keys():
        q_gender = df_3.loc[q]['gender'][0]
        genders.append(labels.loc[q_gender]['Label'])
    speakerGender.append(genders)

KeyboardInterrupt: ignored

In [ ]:
# Add gender column
df_quotesNY.insert(3, 'gender', speakerGender)

### Old examples

In [ ]:
path_to_file = '/content/drive/MyDrive/speakers.json' 

with bz2.open(path_to_file, 'rb') as s_file:
  for instance in s_file:
    instance = json.loads(instance) # loading a sample
    print(instance) 

In [ ]:
# Loading of the quotebank dataset

path_to_file = '/content/drive/MyDrive/Quotebank/quotes-2020.json.bz2' 

with bz2.open(path_to_file, 'rb') as s_file:
  for instance in s_file:
    instance = json.loads(instance) # loading a sample
    print(instance) 